In [668]:
import pandas as pd
import numpy as np
import natasha
import razdel
import re
import string
import annoy
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
   
    PER,
    Doc
)
from gensim.models import FastText, word2vec
import nltk
from nltk.corpus import stopwords
stopwords_ru = stopwords.words('russian')
nltk.download('stopwords')
from pymorphy2 import MorphAnalyzer

[nltk_data] Downloading package stopwords to /home/medic/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

In [314]:
from corus import load_lenta
path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)
#next(records)

In [17]:
count_records = 0
for _ in records:
    count_records += 1
print(count_records)

739338


In [44]:
# create df with texts
lenta_ru = pd.DataFrame(columns=['tags', 'title', 'topic', 'text'])
new_rows = []

for _ in records:
    article = next(records)
    #print(article.title)
    new_row = {}
    new_row = {'tags': article.tags,
               'title': article.title,
               'topic': article.topic,
               'text': article.text}
    new_rows.append(new_row)

lenta_ru = pd.DataFrame(new_rows)
lenta_ru

,tags,title,topic,text
0,Мир,Обнаружено самое счастливое место на планете,Путешествия,Сотрудники социальной сети Instagram проанализ...
1,Общество,Хакеры рассказали о планах Великобритании зами...,Мир,Хакерская группировка Anonymous опубликовала н...
2,Coцсети,Российскую молодежь предложили обучать духовны...,Интернет и СМИ,Российская молодежь лучше усвоит духовные ценн...
3,Политика,Полет российских бомбардировщиков над Карибски...,Мир,Минобороны опубликовало на YouTube видео полет...
4,Политика,Британцам придется заплатить за путешествия по...,Мир,Жители Великобритании будут платить семь евро ...
...,...,...,...,...
369670,Все,Литва засудила участников переворота 91 года,Мир,Россия крайне негативно оценивает судебный про...
369671,Все,Киргизия ведет бои на границах с Таджикистаном...,Мир,"На юге Киргизии, а именно в Баткенском и Чон-А..."
369672,Все,Южно-Сахалинск объявлен очагом холеры,Россия,Сегодня областной центр Сахалина и Курил получ...
369673,Все,В горах Дагестана идут активные боевые действия,Россия,Сегодня утром в районах дагестанских селений Ч...


In [140]:
# method for cleaning text
morph = MorphAnalyzer()
def prep_text(text):
    text = text.lower()
    text = re.sub(r'@[\w]*', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'[\d]', '', text)
    text = ' '.join([word for word in text.split() if word not in stopwords_ru])
    output_text = []
    for token in text.split():
        token = morph.normal_forms(token)[0]
        output_text.append(token)
    
    return output_text

In [614]:
# testing with real data
text = prep_text(lenta_ru['title'][0])
text

['обнаружить', 'самый', 'счастливый', 'место', 'планета']

In [272]:
lenta_ru['lemmatize_title'] = lenta_ru.apply(lambda x: prep_text(x['title']),  axis = 1)

In [273]:
lenta_ru

,tags,title,topic,text,lemmatize_title,vector_title,lemmaatize_title_tags_topic
0,Мир,Обнаружено самое счастливое место на планете,Путешествия,Сотрудники социальной сети Instagram проанализ...,"[обнаружить, самый, счастливый, место, планета]",-0.021758,"[мирпутешествие, обнаружить, самый, счастливый..."
1,Общество,Хакеры рассказали о планах Великобритании зами...,Мир,Хакерская группировка Anonymous опубликовала н...,"[хакер, рассказать, план, великобритания, зами...",0.033977,"[обществомиро, хакер, рассказать, план, велико..."
2,Coцсети,Российскую молодежь предложили обучать духовны...,Интернет и СМИ,Российская молодежь лучше усвоит духовные ценн...,"[российский, молодёжь, предложить, обучать, ду...",-0.003793,"[coцсетиинтернуть, сми, российский, молодёжь, ..."
3,Политика,Полет российских бомбардировщиков над Карибски...,Мир,Минобороны опубликовало на YouTube видео полет...,"[полёт, российский, бомбардировщик, карибский,...",-0.004691,"[политикамир, полёт, российский, бомбардировщи..."
4,Политика,Британцам придется заплатить за путешествия по...,Мир,Жители Великобритании будут платить семь евро ...,"[британец, прийтись, заплатить, путешествие, е...",-0.001439,"[политикамир, британец, прийтись, заплатить, п..."
...,...,...,...,...,...,...,...
369670,Все,Литва засудила участников переворота 91 года,Мир,Россия крайне негативно оценивает судебный про...,"[литва, засудить, участник, переворот, год]",-0.011858,"[всемиро, литва, засудить, участник, переворот..."
369671,Все,Киргизия ведет бои на границах с Таджикистаном...,Мир,"На юге Киргизии, а именно в Баткенском и Чон-А...","[киргизия, вести, бой, граница, таджикистан, у...",-0.001826,"[всемиро, киргизия, вести, бой, граница, таджи..."
369672,Все,Южно-Сахалинск объявлен очагом холеры,Россия,Сегодня областной центр Сахалина и Курил получ...,"[южносахалинск, объявить, очаг, холера]",-0.009875,"[всероссия, южносахалинск, объявить, очаг, хол..."
369673,Все,В горах Дагестана идут активные боевые действия,Россия,Сегодня утром в районах дагестанских селений Ч...,"[гора, дагестан, идти, активный, боевой, дейст...",0.004312,"[всероссия, гора, дагестан, идти, активный, бо..."


In [682]:
# save to picke lenta_ru
lenta_ru.to_pickle('lenta_ru_w2c.pickle')

### Работа с моделью word2vec
Построим модель которая будет показывать n ближайших сосетей из списка статей сайта lenta.ru 
Будем использовать модель word2vec и библиотеку annoy для писка ближайх векторов из обученной модели.

In [585]:
w2v_lenta = Word2Vec(vector_size=300,
                       window = 5,
                       min_count = 1,
                       workers = 32,
                       seed = 34)

In [586]:
# create vocabulary with words in titles of articles
w2v_lenta.build_vocab(lenta_ru['lemmatize_title'])

In [587]:
# train model word2vec
w2v_lenta.train(lenta_ru['lemmatize_title'], total_examples=len(lenta_ru['lemmatize_title']), epochs=100)

(154600470, 157697820)

In [588]:
# list of word in model
w2v_lenta.wv.key_to_index

{'россия': 0,
 'российский': 1,
 'сша': 2,
 'москва': 3,
 'новый': 4,
 'год': 5,
 'назвать': 6,
 'человек': 7,
 'суд': 8,
 'доллар': 9,
 'украина': 10,
 'стать': 11,
 'изз': 12,
 'путин': 13,
 'дело': 14,
 'предложить': 15,
 'миллион': 16,
 'отказаться': 17,
 'американский': 18,
 'свой': 19,
 'погибнуть': 20,
 'задержать': 21,
 'рубль': 22,
 'первый': 23,
 'президент': 24,
 'получить': 25,
 'глава': 26,
 'бывший': 27,
 'россиянин': 28,
 'мир': 29,
 'тысяча': 30,
 'миллиард': 31,
 'рассказать': 32,
 'обвинить': 33,
 'московский': 34,
 'сборная': 35,
 'военный': 36,
 'самолёт': 37,
 'объявить': 38,
 'наслать': 39,
 'дом': 40,
 'убийство': 41,
 'признать': 42,
 'найти': 43,
 'власть': 44,
 'арестовать': 45,
 'британский': 46,
 'банк': 47,
 'самый': 48,
 'матч': 49,
 'запретить': 50,
 'взрыв': 51,
 'выборы': 52,
 'решить': 53,
 'против': 54,
 'украинский': 55,
 'полиция': 56,
 'два': 57,
 'боевик': 58,
 'правительство': 59,
 'появиться': 60,
 'оказаться': 61,
 'выйти': 62,
 'начать': 63,
 

In [590]:
# check what is work
w2v_lenta.wv.most_similar(positive=['полёт', 'российский', 'бомбардировщик', 'сша'])

[('россия', 0.5135189294815063),
 ('b', 0.4757726490497589),
 ('f', 0.46856045722961426),
 ('истребитель', 0.4581150412559509),
 ('тум', 0.44421136379241943),
 ('сша', 0.4432278275489807),
 ('росавиация', 0.4241088926792145),
 ('минобороны', 0.42266926169395447),
 ('фа', 0.41500526666641235),
 ('палубный', 0.41426151990890503)]

In [621]:
# create array vectors 
index_w2v = annoy.AnnoyIndex(300 ,'angular')

for i, sent in enumerate(lenta_ru['lemmatize_title']):
    res1 = [w2v_lenta.wv[w] for w in sent if w in w2v_lenta.wv.key_to_index.keys()]
    v_w2v = sum(res1) / len(res1) if res1 else np.zeros(300)
    index_w2v.add_item(i, v_w2v)

index_w2v.build(20)
index_w2v.get_n_items()

True

In [662]:
# getting vec for any text
def get_vec_for_annoy(text):
    res1 = [w2v_lenta.wv[w] for w in text if w in w2v_lenta.wv.key_to_index.keys()]
    v_w2v = sum(res1) / len(res1) if res1 else np.zeros(300)
    return v_w2v

In [663]:
# function for get articles by text, number - count of getting aticles
def get_news_by_text(text, numbers = 5):
    text = prep_text(text)
    row_indexs = index_w2v.get_nns_by_vector(get_vec_for_annoy(text), numbers)
    df_nns = lenta_ru.iloc[row_indexs]
    articles = list(df_nns['text'])
    
    for article in articles:
        print("-"*64)
        print(article)    

In [680]:
text = 'российское правительство намерено понизить налоги'
get_news_by_text(text, 4)

----------------------------------------------------------------
Министерство финансов отказалось от планов ввести налог на выплаты российских компаний по еврооблигациям в адрес иностранных структур. Об этом сообщает «Интерфакс» со ссылкой на главу ведомства Антона Силуанова. Обложить налогами выплаты по евробондам в адрес иностранных компаний Министерство финансов предложило в январе текущего года. Речь идет о выплатах в адрес SPV-компаний (special purpose vehicle), то есть фактически российских корпоративных эмитентов евробондов. Деловые СМИ писали, что если такое решение будет принято, заимствование денег на зарубежных рынках будет затруднено. Сколько государство могло бы заработать за счет изменения налогового законодательства, не сообщалось. SPV-компании являются посредниками по выпуску облигаций, специально созданными для этой цели. С помощью посредника российские компании добиваются того, что иностранные покупатели их долговых бумаг не платят налоги с купонных доходов. В первом 

### Выводы по модели word2vec 
Данная модель вышла с свет в далеком 2014 году, и честно говоря данный подход уже не такой прорывной как казался тогда, тем не менее он работает до сих пор и используется в работе благодаря своей простоте.
Из минусов стоить заметить что подход несколько груботат, и не всегда выдает нужный результат. 
Однако, стоить заметить, при должной настройке всех параметров модели и хорошем датасете можно достичь очень хороших результатов.

In [667]:
ft_lenta = FastText(vector_size=300,
                    window = 5,
                    min_count = 1,
                    workers = 32,
                    seed = 34)

In [670]:
#w2v_lenta.build_vocab(lenta_ru['lemmatize_title'])
ft_lenta.build_vocab(lenta_ru['lemmatize_title'])

In [671]:
ft_lenta.train(lenta_ru['lemmatize_title'], total_examples=len(lenta_ru['lemmatize_title']), epochs=100)

(220858210, 225282600)

In [672]:
# check what is work
ft_lenta.wv.most_similar(positive=['полёт', 'российский', 'бомбардировщик', 'сша'])

[('российсконикарагуанский', 0.6975217461585999),
 ('российскокорейский', 0.6913001537322998),
 ('американороссийский', 0.6792899370193481),
 ('российскокитайский', 0.6774388551712036),
 ('российскоизраильский', 0.6703558564186096),
 ('российскоиндийский', 0.6700694561004639),
 ('российскоиранский', 0.6662083864212036),
 ('российскосирийский', 0.6660041213035583),
 ('российскобританский', 0.661270797252655),
 ('российскоамериканский', 0.6603701710700989)]

In [674]:
# create array vectors 
index_ft = annoy.AnnoyIndex(300 ,'angular')

for i, sent in enumerate(lenta_ru['lemmatize_title']):
    res1 = [ft_lenta.wv[w] for w in sent if w in ft_lenta.wv.key_to_index.keys()]
    v_ft = sum(res1) / len(res1) if res1 else np.zeros(300)
    index_ft.add_item(i, v_ft)

index_ft.build(20)
index_ft.get_n_items()

369675

In [675]:
# getting vec for any text
def get_vec_for_annoy_ft(text):
    res1 = [ft_lenta.wv[w] for w in text if w in ft_lenta.wv.key_to_index.keys()]
    v_ft = sum(res1) / len(res1) if res1 else np.zeros(300)
    return v_ft

In [676]:
# function for get articles by text, number - count of getting aticles
def get_news_by_text_ft(text, numbers = 5):
    text = prep_text(text)
    row_indexs = index_ft.get_nns_by_vector(get_vec_for_annoy_ft(text), numbers)
    df_nns = lenta_ru.iloc[row_indexs]
    articles = list(df_nns['text'])
    
    for article in articles:
        print("-"*64)
        print(article)   

In [681]:
text = 'российское правительство намерено понизить налоги'
get_news_by_text_ft(text, 4)

----------------------------------------------------------------
Правительство России обсудило возможность изменения налогов на одном из первых после выборов совещаний у премьер-министра страны Дмитрия Медведева. Об этом пишет принадлежащая семье Демьяна Кудрявцева газета «Ведомости» со ссылкой на источники. По данным собеседников издания, в ходе встречи поднимались вопросы о повышении НДФЛ на два процентных пункта, до 15 процентов, с введением необлагаемого минимума. Как отмечает газета, ее собеседники расходятся в трактовке итогов совещания — один источник говорит о том, что решение окончательно не принято, другой заявляет, что «консенсус достигнут». Газета пишет, что в правительстве осенью 2017 года на совещаниях уже анализировали последствия повышения НДФЛ до 15-16 процентов, а также введение необлагаемого минимума. На совещании также поднимались вопросы о проведении двух налоговых маневров. Так, по словам источников, для нефтяников предлагается обнулить экспортную пошлину при повы

### Выводы по модели FastText
Fasttext является модификацией модели word2vec, использует в своей работе отличный от word2vec подход.
В составлении словаря используется не слово а ngramы - части слова.
На предложенных задачах word2vec обе модели показали себя одинаково хорошо. 
Я нее могу выделить одну из моделей.